In [1]:
pasadena_osm = open("pasadena_openstreetmap.osm", "r")
example_osm = open("example.osm", "r")

In [2]:
# FIND WHAT TAGS EXIST IN THE MAP FILE BUT ALSO THE COUNT OF EACH TAG

import xml.etree.cElementTree as ET
import pprint
import re

def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename):
        if elem.tag in tags.keys():
            tags[elem.tag]+= 1
        else:
            tags[elem.tag] = 1
    return tags

count_tags(pasadena_osm)

{'bounds': 1,
 'member': 6891,
 'nd': 1311359,
 'node': 1183432,
 'osm': 1,
 'relation': 881,
 'tag': 739625,
 'way': 116377}

In [3]:
# EXPLORE THE "K" VALUE FOR EACH "<tag>" AND SEE IF THERE ARE ANY POTENTIAL PROBLEMS

import xml.etree.cElementTree as ET
import pprint
import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        k = element.attrib['k']
        if re.search(lower,k):
            keys["lower"] += 1
        elif re.search(lower_colon,k):
            keys["lower_colon"] += 1
        elif re.search(problemchars,k):
            keys["problemchars"] += 1
        else:
            keys["other"] += 1
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

keys = process_map('pasadena_openstreetmap.osm')
pprint.pprint(keys)



{'lower': 431676, 'lower_colon': 305609, 'other': 2340, 'problemchars': 0}


In [4]:
# FIND THE NUMBER OF UNIQUE CONTRIBUTORS

import xml.etree.cElementTree as ET
import pprint
import re

def get_user(element):
    return element.get('uid')


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        x = get_user(element)
        if x not in users and x != None:
            users.add(x)
    return users


users = process_map('pasadena_openstreetmap.osm')
len(users)
#pprint.pprint(users)

449

In [5]:
# AUDIT STREET NAMES: THIS INCLUDES AUDITING THE DATA FOR IRREGULAR/WEIRD STREET TYPES

import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "pasadena_openstreetmap.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { 
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Dr": "Drive",
            "Dr.": "Drive",
            "Ln": "Lane",
            "Pkwy": "Parkway",
            "Rd.": "Road",
            "Rd": "Road",
            "St": "Street",
            "St.": "Street"
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

def update_name(name, mapping):
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping.keys():
            #print 'Before: ' , name
            name = re.sub(m.group(), mapping[m.group()], name)
            #print 'After: ', name
    return name

# FIND AND PRINT OUT THE ACTUAL ARRAYS OF IRREGULAR DATA
st_types = audit(OSMFILE)
pprint.pprint(dict(st_types))

# PRINT THE BETTER NAME
def cleanup_name():
    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_name(name, mapping)
            print (name, "=>", better_name)
            
cleanup_name()


{'103': set(['Vista Ave, Suite 103']),
 '106': set(['South Lake Ave #106']),
 '38.3': set(['LA 2 PM 38.3']),
 '52': set(['Avenue 52']),
 '53': set(['N Avenue 53']),
 '56': set(['N Avenue 56']),
 '61': set(['North Avenue 61']),
 '64': set(['Avenue 64', 'North Avenue 64']),
 '702': set(['Duarte Road #702']),
 'Alley': set(['Martin Alley', 'Miller Alley']),
 'Ave': set(['850 S Baldwin Ave',
             'Arcadia Ave',
             'Camino Real Ave',
             'Crown Ave',
             'E Naomi Ave',
             'Fair Oaks Ave',
             'Fairview Ave',
             'Harkness Ave',
             'Holliston Ave',
             'Lemon Ave',
             'Lincoln Ave',
             'Longden Ave',
             'Michigan Ave',
             'N Allen Ave',
             'N Raymond Ave',
             'Naomi Ave',
             'North Fair Oaks Ave',
             'S Baldwin Ave',
             'S Lake Ave',
             'South Catalina Ave',
             'South Fair Oaks Ave',
             'Wind

In [7]:
# AUDIT PHONE NUMBERS AND CLEAN THEM

import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "pasadena_openstreetmap.osm"

def audit_phone_type(phone_types, phone_name):
    phone_types.add(phone_name)
    
def is_phone_number(elem):
    return (elem.attrib['k'] == "phone")


def audit_phone(osmfile):
    osm_file = open(osmfile, "r")
    phone_types = set()
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_phone_number(tag):
                    audit_phone_type(phone_types, tag.attrib['v'])
    osm_file.close()
    return phone_types

def update_phones(phonenumber):
    # remove non-digit characters
    phonenumber = ''.join(ele for ele in phonenumber if ele.isdigit())

    if len(phonenumber) == 10:
        phonenumber =  ('{0}-{1}-{2}'.format(phonenumber[:3], phonenumber[3:6], phonenumber[6:]))
    elif phonenumber.startswith('1'):
        phonenumber = ('{0}-{1}-{2}'.format(phonenumber[1:4], phonenumber[4:7], phonenumber[7:]))
    elif phonenumber.startswith('01'):
        phonenumber = phonenumber[2:]
    elif len(phonenumber) > 10:
        phonenumber = ('{0}-{1}-{2}'.format(phonenumber[:3], phonenumber[3:6], phonenumber[6:10]))
    else:
        # this depends on what you have seen in your audit, 
        # and your test run of the cleaning code. Is 'else' required ?
        pass

    return phonenumber

ph_list = audit_phone(OSMFILE)

# TO TEST THE CLEANING FUNCTION

for phone_number in ph_list:
    print (phone_number, '==>',)
    phone_number = update_phones(phone_number)
    print (phone_number)

('(626) 440-0309', '==>')
626-440-0309
('626.356.9460', '==>')
626-356-9460
('(626) 844-0592', '==>')
626-844-0592
('(626) 568-9310', '==>')
626-568-9310
('(626) 578-1818', '==>')
626-578-1818
('+1 626-793-7745', '==>')
626-793-7745
('626-792-5829', '==>')
626-792-5829
('626.356.9889', '==>')
626-356-9889
('(626) 356-7529', '==>')
626-356-7529
('+1 626-844-7008', '==>')
626-844-7008
('+1 626-844-0054', '==>')
626-844-0054
('(818) 790-8900', '==>')
818-790-8900
('(800)-532-8346', '==>')
800-532-8346
('+1 626-796-3030', '==>')
626-796-3030
('+1 626 7745342', '==>')
626-774-5342
('6263194550', '==>')
626-319-4550
('626-449-5172', '==>')
626-449-5172
('626.396.0814', '==>')
626-396-0814
('(626) 287-9671', '==>')
626-287-9671
('626-449-3470', '==>')
626-449-3470
('626-449-8314', '==>')
626-449-8314
('+1 (626).844.0239', '==>')
626-844-0239
('(323)340-8525', '==>')
323-340-8525
('(626) 795-3793', '==>')
626-795-3793
('626.792.2343', '==>')
626-792-2343
('323-255-6456', '==>')
323-255-6456
('

In [8]:
# AUDIT ZIP CODES AND CLEAN THEM

import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "pasadena_openstreetmap.osm"

def audit_zip_type(zip_types, zip_name):
    zip_types.add(zip_name)
    
def is_zip_code(elem):
    return (elem.attrib['k'] == "addr:postcode")


def audit_zip(osmfile):
    osm_file = open(osmfile, "r")
    zip_types = set()
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zip_code(tag):
                    audit_zip_type(zip_types, tag.attrib['v'])
    osm_file.close()
    return zip_types

def update_zipcodes(zipnumber):
    # remove non-digit characters
    zipnumber = ''.join(ele for ele in zipnumber if ele.isdigit())

    if zipnumber.startswith('5'):
        zipnumber =  ('{0}'.format(zipnumber[4:]))
    elif len(zipnumber) > 5:
        zipnumber =  ('{0}'.format(zipnumber[:5]))
    else:
        # this depends on what you have seen in your audit, 
        # and your test run of the cleaning code. Is 'else' required ?
        pass

    return zipnumber

zip_list = audit_zip(OSMFILE)

# TO TEST THE CLEANING FUNCTION

for zip_number in zip_list:
    print (zip_number, '==>',)
    zip_number = update_zipcodes(zip_number)
    print (zip_number)

('90042-4229', '==>')
90042
('CA 91101', '==>')
91101
('91780', '==>')
91780
('91102', '==>')
91102
('91103', '==>')
91103
('91101', '==>')
91101
('91106', '==>')
91106
('91107', '==>')
91107
('91104', '==>')
91104
('91105', '==>')
91105
('91108', '==>')
91108
('91109', '==>')
91109
('91125', '==>')
91125
('90041-1238', '==>')
90041
('91775', '==>')
91775
('91024', '==>')
91024
('91001', '==>')
91001
('91007', '==>')
91007
('91006', '==>')
91006
('91182', '==>')
91182
('91105-1825', '==>')
91105
('91011', '==>')
91011
('CA 91775', '==>')
91775
('91030', '==>')
91030
('90041', '==>')
90041
('90042', '==>')
90042
('5101 York Blvd, Los Angeles, CA 90042', '==>')
90042


In [9]:
# SHAPE THE DATA FROM XML FORMAT AND EXPORT TO CSV

import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "pasadena_openstreetmap.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields = NODE_FIELDS, way_attr_fields = WAY_FIELDS, problem_chars = PROBLEMCHARS, default_tag_type = 'regular'):
# Clean and shape node or way XML element to Python dict
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = [] # Handle secondary tags the same way for both node and way elements

# MY CODE
    if element.tag == 'node':
        for attribute in element.attrib:
            if attribute in NODE_FIELDS:
                node_attribs[attribute] = element.attrib[attribute]
        for child in element:
            nodes = {}
            if PROBLEMCHARS.match(child.attrib['k']):
                pass
            elif LOWER_COLON.match(child.attrib['k']):
                nodes["type"] = child.attrib['k'].split(":", 1)[0]
                nodes["key"] = child.attrib['k'].split(":", 1)[1]
                nodes["id"] = element.attrib['id']
                nodes["value"] = child.attrib['v']
                if child.attrib["k"] == 'addr:street':
                    nodes["value"] = update_name(child.attrib["v"], mapping)
                if child.attrib["k"] == 'addr:postcode':
                    nodes["value"] = update_zipcodes(child.attrib["v"])
                tags.append(nodes)
            else:
                nodes["type"] = "regular"
                nodes["key"] = child.attrib['k']
                nodes["id"] = element.attrib['id']
                nodes["value"] = child.attrib['v']
                tags.append(nodes)
                if child.attrib["k"] == 'phone':
                    nodes["value"] = update_phones(child.attrib["v"])
                tags.append(nodes)
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for attribute in element.attrib:
            if attribute in WAY_FIELDS:
                way_attribs[attribute] = element.attrib[attribute]
                position = 0
        for child in element:
            way_n = {}
            way_t = {}
            if child.tag == "nd":
                if element.attrib["id"] not in way_nodes:
                    way_n["position"] = position
                    way_n["id"] = element.attrib["id"]
                    way_n["node_id"] = child.attrib["ref"]
                    way_nodes.append(way_n)
                    position += 1
                else:
                    way_n["position"] += 1
                    way_n["id"] = element.attrib["id"]
                    way_n["node_id"] = child.attrib["ref"]
                    way_nodes.append(way_n)
            elif child.tag == "tag":
                if PROBLEMCHARS.match(child.attrib["k"]):
                    pass
                elif LOWER_COLON.match(child.attrib["k"]):
                    way_t["type"] = child.attrib["k"].split(":",1)[0]
                    way_t["key"] = child.attrib["k"].split(":",1)[1]
                    way_t["id"] = element.attrib["id"]
                    way_t["value"] = child.attrib["v"]
                    if child.attrib["k"] == 'addr:street':
                        way_t["value"] = update_name(child.attrib["v"], mapping)
                    if child.attrib["k"] == 'addr:postcode':
                        way_t["value"] = update_zipcodes(child.attrib["v"])
                    tags.append(way_t)
                else:
                    way_t["type"] = "regular"
                    way_t["key"] = child.attrib["k"]
                    way_t["id"] = element.attrib["id"]
                    way_t["value"] = child.attrib["v"]
                    if child.attrib["k"] == 'phone':
                        way_t["value"] = update_phones(child.attrib["v"])
                    tags.append(way_t)
        return {'way' : way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


process_map(OSM_PATH, validate=True)

In [ ]:
#SQL QUERIES

# TO FIND UNIQUE USERS
QUERY = '''
SELECT COUNT(DISTINCT(x.uid))
    FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) x;

'''

# TOP 10 AMENITIES

QUERY = '''
SELECT value, COUNT(*) as num
    FROM (SELECT * FROM ways_tags
        UNION ALL
        SELECT * FROM nodes_tags)
    WHERE key='amenity'
    GROUP BY value
    ORDER BY num DESC
    LIMIT 10;

'''

# TOP 10 CUISINES

QUERY = '''
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant' or value='fast_food') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='cuisine'
GROUP BY nodes_tags.value
ORDER BY num DESC;

'''

# FIND NUMBER OF NODES

import sqlite3

db = sqlite3.connect("pas_openstreetmap.db")
c = db.cursor()

QUERY = '''
SELECT COUNT(*) FROM nodes;

'''
c.execute(QUERY)
run = c.fetchall()

print (run)

# FIND NUMBER OF WAYS

import sqlite3

db = sqlite3.connect("pas_openstreetmap.db")
c = db.cursor()

QUERY = '''
SELECT COUNT(*) FROM nodes;

'''
c.execute(QUERY)
run = c.fetchall()

print (run)

# FIND NUMBER OF SCHOOLS
QUERY = '''
SELECT value, COUNT(*) as num
    FROM (SELECT * FROM ways_tags
        UNION ALL
        SELECT * FROM nodes_tags)
    WHERE key='amenity'
    AND (value='school' or value='college' or value='kindergarten' or value='university')
    GROUP BY Value
    ORDER BY num DESC;

'''